In [162]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [163]:
df = pd.read_csv("../data/train.csv")

In [164]:
df = df.drop(["Unnamed: 0"],axis=1)
df = df.set_index("id")
label = df["stroke"]
df = df.drop(["stroke"],axis=1)

In [165]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
id,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked


In [166]:
df.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  177
smoking_status         0
dtype: int64

In [167]:
df["bmi"].mode()

0    26.7
1    27.7
Name: bmi, dtype: float64

In [168]:
# Find mode value 
mode_1 = df["bmi"].mode()[0]
joblib.dump(mode_1,"../model_clf/mode")


['../model_clf/mode']

In [169]:
mode_1 = df["bmi"].mode()[1]
joblib.dump(mode_1,"../model_tree/mode")

['../model_tree/mode']

In [170]:
def missing_values(df):
    
    mode = joblib.load("../model_tree/mode")
    df = df.fillna(mode)
    return df
    
    

In [171]:
def fit_one_hot(df):
    encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
    encoder.fit(df[["gender","ever_married","work_type","Residence_type"]])
    joblib.dump(encoder,"../model_tree/encoder")

In [172]:
def transform_one_hot(df):
    encoder = joblib.load("../model_tree/encoder")
    result = encoder.transform(df[["gender","ever_married","work_type","Residence_type"]])
    df = df.drop(["gender","work_type","Residence_type","ever_married"],axis=1)
    df[encoder.get_feature_names_out()] = result
    return df

In [173]:
def fit_ordinal(df):
    encoder = OrdinalEncoder(handle_unknown="use_encoded_value",unknown_value=4)
    encoder.fit(df[["smoking_status"]])
    joblib.dump(encoder,"../model_clf/encoder_ordinal")

In [174]:
def transform_ordinal(df):
    encoder = joblib.load("../model_clf/encoder_ordinal")
    result = encoder.transform(df[["smoking_status"]])
    df[encoder.get_feature_names_out()] = result
    return df

In [175]:
def std_scaler_fit(df):
    scaler = StandardScaler()
    scaler.fit(df)
    joblib.dump(scaler,"../model_tree/scaler")

In [176]:
def std_scaler_transform(df):
    scaler = joblib.load("../model_tree/scaler")
    result = scaler.transform(df)
    df.iloc[:,:] = result
    return df

In [177]:
def train_model_tree(df,label):
    x_train, x_test, y_train, y_test = split_test_train(df,label)
    clf = DecisionTreeClassifier()
    clf.fit(x_train,y_train)
    joblib.dump(clf,"../model_tree/tree")
    return clf.score(x_test,y_test)
    

In [178]:
def split_test_train(df,label):
    X = df
    Y = label
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    return x_train, x_test, y_train, y_test
    

In [179]:
def pipeline(df,label):
    df = missing_values(df)
    
    fit_one_hot(df)
    df = transform_one_hot(df)
    
    fit_ordinal(df)
    df = transform_ordinal(df)
    
    std_scaler_fit(df)
    df = std_scaler_transform(df)
    score = train_model_tree(df,label)
    return score
    
    
    

In [180]:
pipeline(df, label)

0.8814814814814815